In [1]:
import sys
import rospy
import moveit_commander
import copy
import moveit_msgs.msg
import geometry_msgs.msg
from math import pi
from tf.transformations import *
from moveit_msgs.msg import RobotState, Constraints, JointConstraint, OrientationConstraint

In [2]:

rospy.init_node('test_ur', anonymous=True)
moveit_commander.roscpp_initialize(sys.argv)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

robot.get_group_names()

['arm', 'gripper']

In [3]:
group_name = robot.get_group_names()
move_group = moveit_commander.MoveGroupCommander(group_name[0])

In [4]:
planning_frame = move_group.get_planning_frame()

In [5]:
planning_frame

'world'

In [6]:
eef_link = move_group.get_end_effector_link()

In [7]:
eef_link

'robotiq_85_base_link'

In [21]:

robot.get_current_state()


joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint, robotiq_85_left_knuckle_joint]
  position: [1.3419325545567524, 0.971414594415859, -1.7649813508366536, 0.7708403460877733, 1.3443759669816613, 0.00723850112264568, 8.203212740198751e-06]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False

In [9]:

move_group.set_named_target("up")
plan = move_group.plan()

In [10]:
move_group.execute(plan, wait=True)

True

In [11]:

def move_joints(move_group, goal):
    move_group.go(goal, wait=True)
    move_group.stop()
    
def get_joint_state(move_group):
    joint_state = move_group.get_current_joint_values()
    print "---- rad value ----"
    print joint_state
    print "---- degree value ---- "
    print [joint*180/math.pi for joint in joint_state]

In [12]:

import math

get_joint_state(move_group)

---- rad value ----
[-0.00018850486171473335, -1.2665856066614953, 4.6044316061255586e-05, -7.159822560698359e-05, 0.00011412394868504094, 5.7462574043043446e-05]
---- degree value ---- 
[-0.010800532993951435, -72.57000965372065, 0.002638144980876375, -0.004102276147905657, 0.006538820601020427, 0.0032923629726243847]


In [13]:
goal = [ n * math.pi / 180 for n in [90,-100,90,-100,-90,0]]

move_joints(move_group, goal)
get_joint_state(move_group)

---- rad value ----
[1.570706389656567, -1.7454117487793406, 1.5707096866816972, -1.7453226877274117, -1.570698068381688, 0.00030053778135208375]
---- degree value ---- 
[89.99484698155221, -100.00472671760453, 89.99503588717715, -99.99962389521001, -89.9943702076215, 0.017219546455699935]


In [14]:
# cartesin value retrieval

def get_goal_pose(move_group):
    joint_state = move_group.get_current_pose()
    print joint_state

In [22]:

get_goal_pose(move_group)

header: 
  seq: 0
  stamp: 
    secs: 420
    nsecs: 914000000
  frame_id: "world"
pose: 
  position: 
    x: 0.011200572635
    y: 0.618054634947
    z: 0.925226089753
  orientation: 
    x: -0.707896548158
    y: -0.706228565848
    z: -0.00708547884554
    w: 0.00857239034082


In [35]:

import geometry_msgs.msg

pose_goal = geometry_msgs.msg.Pose()

pose_goal.position.x = 0.0
pose_goal.position.y = 0.65
pose_goal.position.z = 1.37679003904

pose_goal.orientation.x = 0.768049136044
pose_goal.orientation.y = -0.00683168224924
pose_goal.orientation.z = 0.00497378503076
pose_goal.orientation.w = 0.640335157712
move_group.set_pose_target(pose_goal)

아래 있는 cell 이 quaternion orientation 이 제대로 잡혀 있음... 여기에서 contraint 를 한번 걸어보는 것으로 해보기 ... 

In [67]:
pose_goal.position.x = 0.44
pose_goal.position.y = 0.45
pose_goal.position.z = 0.998 - 0.376

pose_goal.orientation.x = -0.707896548158
pose_goal.orientation.y = -0.706228565848
pose_goal.orientation.z = 0.00708547884554
pose_goal.orientation.w = 0.00857239034082
move_group.set_pose_target(pose_goal)

plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()

ORIENTATION CONSTRAINT 넣어주기 

In [54]:


constraints = Constraints()

constraints.name = "constraints1"

oc1 = OrientationConstraint()

# input orientation constraints
constraints.orientation_constraints.append(oc1)
move_group.set_path_constraints(constraints)


pose_goal.position.x = 0.40
pose_goal.position.y = 0.44
pose_goal.position.z = 0.998 - 0.376

pose_goal.orientation.x = 0.768049136044
pose_goal.orientation.y = -0.00683168224924
pose_goal.orientation.z = 0.00497378503076
pose_goal.orientation.w = 0.640335157712
move_group.set_pose_target(pose_goal)

plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()



In [1]:

import tf

tmp = tf.transformations.euler_from_quaternion([
    pose_goal.orientation.x,
    pose_goal.orientation.y,
    pose_goal.orientation.z,
    pose_goal.orientation.w])

[each*180/math.pi for each in tmp]

NameError: name 'pose_goal' is not defined

In [2]:
tf.transformations.quaternion_from_euler(30*math.pi/180,
                                         60*math.pi/180,
                                         90*math.pi/180)

NameError: name 'math' is not defined

In [117]:
[p,q,r] = [each*math.pi/180 for each in [0., 60, 90.]]

x,y,z,w =tf.transformations.quaternion_from_euler(p,q,r)

NameError: name 'tf' is not defined

In [22]:
move_group.set_pose_target(pose_goal)

plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()